In [68]:
import torch
torch.arange(8).long()

tensor([0, 1, 2, 3, 4, 5, 6, 7])

In [1]:
import jieba
import lancedb
uri = "./data/sample-lancedb"
db = lancedb.connect(uri)

In [2]:
table = db.open_table("medical")

In [52]:
question = "请问冠心病怎么治"

In [53]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('BAAI/bge-base-zh-v1.5')
embeddings_query = model.encode(question, normalize_embeddings=True)

In [54]:
results =  table.search(embeddings_query).limit(3).to_pandas()

In [55]:
results

,query,answer,vector,queryseg,_distance
0,冠心病怎么保养啊？，我有冠心病，有点难治，但是医生说平时留意调理还是可以慢慢治疗的，您好冠心...,你好，有冠心病这个疾病，其实是需要有调理的，尤其是控制饮食，不能够吃太多油腻的食物，也不能够...,"[0.043754194, -0.01398439, -0.008187677, -0.00...",冠心病 怎么 保养 啊 ？ ， 我 有 冠心病 ， 有点 难治 ， 但是 医生 说 平时 留...,0.449909
1,谁能说说冠心病怎么办，我单位的一名同事在健康检查中仔细检查出患冠心病，他很纳闷儿他们家族并没...,影响冠心病的因素有很多，下面这几种因素中，请根据日常生活技术及医学知识推测一下哪个是最危险的...,"[0.030950908, -0.033846293, 0.022763591, -0.01...",谁 能 说 说 冠心病 怎么办 ， 我 单位 的 一名 同事 在 健康检查 中 仔细检查 出...,0.523649
2,行冠心病的辅助治疗有些什么？,心脏康复,"[-0.0121756075, 0.013376501, -0.008880715, 0.0...",行 冠心病 的 辅助 治疗 有些 什么 ？,0.546783


In [56]:
results["q_a"] = results["query"] + ":" + results["answer"]

In [57]:
document = "\n".join(results["q_a"].to_list())

In [61]:

PROMPT  ="""
请根据参考资料回答问题
要求：
1.不要跳出参考资料的范围去回答问题。
2.如果参考资料的内容无法回答问题,则告知无法回答
3.回答尽量简洁明了

参考资料：
{}

问题：
{}

请给出你的回答：
""".format(document,question)

In [62]:
PROMPT

'\n请根据参考资料回答问题\n要求：\n1.不要跳出参考资料的范围去回答问题。\n2.如果参考资料的内容无法回答问题,则告知无法回答\n3.回答尽量简洁明了\n\n参考资料：\n冠心病怎么保养啊？，我有冠心病，有点难治，但是医生说平时留意调理还是可以慢慢治疗的，您好冠心病怎么调理啊？:你好，有冠心病这个疾病，其实是需要有调理的，尤其是控制饮食，不能够吃太多油腻的食物，也不能够吃太多辛辣刺激的食物，而且有冠心病的患者还应该是留意不能够让自己有太大的情绪震荡，也不要让身体太太过疲劳，您平时也可以决定用一些丹参之类的药物泡水。\n谁能说说冠心病怎么办，我单位的一名同事在健康检查中仔细检查出患冠心病，他很纳闷儿他们家族并没有冠心病史，为什么会得冠心病呢？有哪些影响因素呢？:影响冠心病的因素有很多，下面这几种因素中，请根据日常生活技术及医学知识推测一下哪个是最危险的影响因素？一是抽烟，二是喝酒，三是活动，四是情绪激动，冠心病最主要的危险因素首先冠心病是由于冠状动脉粥样硬化、冠状动脉炎或者是冠状动脉痉挛引发的一种临床诊断综合征，它的主要特点是心肌的脑水肿缺氧。生气时情绪激动，容易引发心血管的剧烈疼痛，引起冠心病，因此情绪激动，性格暴躁，是冠心病的重要的危险因素。\n行冠心病的辅助治疗有些什么？:心脏康复\n\n问题：\n请问冠心病怎么治\n\n请给出你的回答：\n'

In [65]:
from openai import OpenAI

client = OpenAI(
    base_url="http://10.193.64.236:8000/v1",
    api_key = "none"
)

chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": PROMPT,
        }
    ],
    max_tokens = 1000,
    model="gpt-3.5-turbo",
)

In [66]:
response = chat_completion.choices[0].message.content

In [67]:
print(response)

根据参考资料，冠心病的治疗方法包括药物治疗和生活方式调整。对于药物治疗，可以选择使用丹参等药物来帮助改善症状。而在生活方式上，需要注意以下几点：

1. 控制饮食：避免过多摄入油腻和辛辣食物，保持低盐、低脂的饮食习惯。
2. 保持良好的作息：保证充足的睡眠，避免过度劳累。
3. 管理情绪：保持心态平和，避免情绪波动过大。
4. 适当运动：进行适量的有氧运动，如散步、慢跑等，以增强心肺功能。
5. 戒烟限酒：避免吸烟和过量饮酒，这对降低冠心病的风险非常重要。

此外，还可以考虑进行心脏康复训练，以提高生活质量并降低病情恶化的风险。总之，冠心病的治疗需要长期坚持，通过药物治疗和生活方式调整相结合，才能有效地控制病情。
